<div style="text-align:center;">
  <img src="images/molssi_main_horizontal.png" style="display: block; margin: 0 auto; max-height:200px;">
</div>


# Manipulating Small Molecules with Python

<div class="alert alert-block alert-info"> 

<h2>Overview</h2>

<strong>Questions</strong>

* How can I load molecular files using RDKit?
* How can I manipulate and create new molecules from an existing molecule?

<strong>Objectives</strong>

* Learn about molecular file formats.
* Modify a ligand file in a meaningful way and save the modified file for future use.

</div>

So far in this workshop, we have learned about using Python for fairly general applications. During the afternoon session, we will use Chemistry-specific libraries with the goal of performing a molecular docking calculation.

In this notebook, we load an "ideal" structure for a small molecule ligand downloaded from the Protein Data Bank and then we manipulate it using a Python library called RDKit.

The approach taken here is manual. We will pick atoms of interest visually, then morph atomic numbers in order to change a carbon atom into a nitrogen atom, and then add a methyl group by changing a hydrogen atom into a carbon atom.

By loading our molecule from an ideal structure with 3D coordinates, we can ensure that we're already near a "good structure" that we can use for docking.

### Libraries for this Notebook

| Library    | Description     |
| :-----------: | :------------ |
| rdkit | Cheminformatics Toolkit |
| Chem | A subset of rdkit for molecule manipulation |
| IPythonConsole | A subset of rdkit to control image quality |
| Draw | A subset of rdkit for structure drawing |
| AllChem | A subset of rdkit for optimizing 3D structures |

After completing this notebook, if you wish to dig deeper on rdkit, consider reading [Getting Started with rdkit in Python](https://www.rdkit.org/docs/GettingStartedInPython.html).

## Molecular File Formats

When working with molecules on a computer, molecules are often represented in a number of different file formats.
As you may conclude from the rest of our notebooks, working with this "zoo" of file formats can sometimes be overwhelming.
Depending on what you are trying to do, you may use a particular file format or a number of different file formats. 
Often, you will have to pick a file format based on the software you are using or the molecular information you want to save. 

In this first notebook, we will work with a ligand called `13U` that binds to the enzyme trypsin.
You can see the structure for the ligand and protein [here](https://www.rcsb.org/structure/2ZQ2).

## Visualizing the Ligand
We have downloaded the ideal structure for this molecule as an SDF (structural data file). 
An SDF typically includes the 3D coordinates of a molecule. 
Let's use a Python library called NGLView to take a look at the 3D structure of our ligand.

In [ ]:
import nglview as nv

view = nv.show_file("ligands/13U.sdf")
view

## Modifying the Ligand Structure
Now that we've visualized our ligand, let's load it into a Python library that allows us to modify it.
We will load this SDF file into RDKit to create an RDKit molecule.
Then, we can modify the molecule and save our new versions.

In [ ]:
from rdkit import Chem # For reading SDF
from rdkit.Chem.Draw import IPythonConsole # So that we can change display settings

# Configuration for displaying in Jupyter notebooks
IPythonConsole.ipython_useSVG = True  # Use SVG for higher quality images
IPythonConsole.drawOptions.addAtomIndices = True  # Show atom indices
IPythonConsole.molSize = 600,600 # Set size of image

ligand = Chem.MolFromMolFile("ligands/13U.sdf")
ligand

In the cell above, we used RDKit to read in the data in the file `ligands/13U.sdf`. 

Our variable `ligand` is an RDKit molecule. We can perform actions on the molecule like getting the number of atoms using functions from RDKit.


In [ ]:
# For example, we can get the number of atoms.
ligand.GetNumAtoms()

We will modify the ligand above, ([13U: N-cyclooctylglycyl-N-(4-carbamimidoylbenzyl)-L-prolinamide](https://www.rcsb.org/ligand/13U)) as our starting point to create two additional ligands. 
In one case, we will substitute nitrogen for carbon in an aromatic ring. In the second case, a carbon will be added to the original aromatic ring. 

Please notice the index numbers attached to each atom in the ligand image generated by the previous cell. This is possible because of an earlier command that we used to display these index numbers, repeated below.

`IPythonConsole.drawOptions.addAtomIndices = True  # Show atom indices`

We will use these index numbers to tell the Python code which atoms to modify.

In [ ]:
# load a duplicate copy of 13U to manipulate
mod_ligand_N = Chem.MolFromMolFile("ligands/13U.sdf")
mod_ligand_N

Using RDKit, we can select a particular atom and change its atomic number.

In [ ]:
# change carbon in ring to a nitrogen
atom = mod_ligand_N.GetAtomWithIdx(23)

atom.SetAtomicNum(7)

mod_ligand_N

In some cases, we might see that morphing an atom leads to an incorrect number of hydrogens on the structure. 
We can explicitly specify the number of hydrogens on an atom. In our case, we would expect for the nitrogen atom in the ring to not have any hydrogens.

In [ ]:
atom.SetNumExplicitHs(0) # Set the number of explicit hydrogens to 0

In [ ]:
# Look at position 23 now
mod_ligand_N

Now we'll perform the same operations, but this time add a methyl group to the ring.
In this notebook, we're going to morph the hydrogen on the carbon atom to a carbon atom.
However, a more "standard" way to do this would be to create two molecule fragments and then merge them together.
To be consistent with our above approach, we'll also morph an atom for this ligand.

In [ ]:
# load another duplicate of the original ligand, but keep the hydrogens

mod_ligand_methyl = Chem.MolFromMolFile("ligands/13U.sdf", removeHs=False)
mod_ligand_methyl        # This is the original structure. In the cells below, we will convert Hydrogen-59 to a Carbon.

In [ ]:
# Use the index number to select the atom we want to change - look at image to see we want to morph atom 59
atom = mod_ligand_methyl.GetAtomWithIdx(59)

atom.SetAtomicNum(6) # Change the atom to carbon
atom.SetNumExplicitHs(3) # Add 3 explicit hydrogens to the carbon

mod_ligand_methyl = Chem.RemoveAllHs(mod_ligand_methyl) # Remove the H's for viewing
mod_ligand_methyl

## Getting reasonable moleculer geometries

Now that we have our manipulated molecules, we'll optimize them using RDKit and save them.

When we say "optimize" in this context, we are referring to a geometry optimization. 
This is a calculation that is done to make sure that the atoms in the molecule are in a reasonable, lower energy position.
Our original ligand is already in an "ideal" geometry, however, we'll perform this step with our modified molecules.

In [ ]:
# Optimize new molecules and save
from rdkit.Chem import AllChem

# Add hydrogens for the optimization
Chem.SanitizeMol(mod_ligand_N)
mod_ligand_NH = Chem.AddHs(mod_ligand_N)


Note here about how hydrogens aren't in the correct place.

In [ ]:
# When you look at this image, it's going to be very confusing!
mod_ligand_NH

In [ ]:
# Do a constrained embedding to keep the ligand in the same position
# this allows for the hydrogens to be added in reasonable locations, but keeps
# the heavy atoms in the same position
# See https://rdkit.org/docs/source/rdkit.Chem.AllChem.html#rdkit.Chem.AllChem.ConstrainedEmbed
constrained_mol = AllChem.ConstrainedEmbed(mod_ligand_NH, mod_ligand_N, useTethers=True)
mod_ligand_NH

In [ ]:
# Perform geometry optimization
opt_N = AllChem.MMFFOptimizeMolecule(mod_ligand_NH)
mod_ligand_NH

In [ ]:
# Repeat process on methyl ligand

mod_ligand_methylH = Chem.AddHs(mod_ligand_methyl)

constrained_mol = AllChem.ConstrainedEmbed(mod_ligand_methylH, mod_ligand_methyl, useTethers=True)
constrained_mol

In [ ]:
opt_methyl = AllChem.MMFFOptimizeMoleculeConfs(mod_ligand_methylH)
Chem.RemoveAllHs(mod_ligand_methylH)

In [ ]:
# save to new files

Chem.MolToMolFile(mod_ligand_NH, 'ligands/13U_modified_N.sdf')
Chem.MolToMolFile(mod_ligand_methylH, 'ligands/13U_modified_methyl.sdf')

In [ ]:
# visualize structures in 3D
view = nv.show_file("ligands/13U_modified_N.sdf")
view

In [ ]:
# visualize structures in 3D
view = nv.show_file("ligands/13U_modified_methyl.sdf")
view

<div class="alert alert-block alert-warning"> 

<h3>Exercise</h3>

Starting with our original ligand, identify one hydrogen you would like to change to a hydroxyl (-OH) group. 
Make the change using RDKit and save your new ligand.

To perform this task, you will need to 

(1) Load a new variable for the ligand to use as a starting point. Make sure the molecule includes hydrogens.  
(2) Select a hydrogen to change and note the index.  
(3) Change the atomic number of the hydrogen.  
(4) Perform a constrained embedding and geometry optimization.

</div>

In [ ]:
ligand = Chem.MolFromMolFile("ligands/13U.sdf", removeHs=False)
ligand

In [ ]:
# Let's change something on the cyclohexane (to the left) to an OH
atom = ligand.GetAtomWithIdx(40)
atom.SetAtomicNum(8)

In [ ]:
ligand

In [ ]:
Chem.SanitizeMol(ligand)

In [ ]:
mod_ligand_OH = Chem.AddHs(ligand)

# Do a constrained embedding to keep the ligand in the same position
# this allows for the hydrogens to be added in reasonable locations, but keeps
# the heavy atoms in the same position
# See https://rdkit.org/docs/source/rdkit.Chem.AllChem.html#rdkit.Chem.AllChem.ConstrainedEmbed
constrained_mol = AllChem.ConstrainedEmbed(mod_ligand_OH, ligand, useTethers=True)
constrained_mol

In [ ]:
opt_OH = AllChem.MMFFOptimizeMoleculeConfs(mod_ligand_OH)
Chem.RemoveAllHs(mod_ligand_OH)

In [ ]:
Chem.MolToMolFile(mod_ligand_OH, 'ligands/13U_modified_OH.sdf')

In [ ]:
view = nv.show_file("ligands/13U_modified_OH.sdf")
view